# Лабораторная работа 4. Классификация. Деревья решений

Яшакин Андрей 6132-010402D



In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree


## 1. Считать датасет



In [51]:
df = pd.read_csv("housing.csv")
df = df.dropna()
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 20433 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20433 non-null  float64
 1   latitude            20433 non-null  float64
 2   housing_median_age  20433 non-null  float64
 3   total_rooms         20433 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20433 non-null  float64
 6   households          20433 non-null  float64
 7   median_income       20433 non-null  float64
 8   median_house_value  20433 non-null  float64
 9   ocean_proximity     20433 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.7+ MB


In [52]:
# Предобработка данных
df = df[['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms',
         'population', 'households', 'median_income', 'median_house_value', 'ocean_proximity']]
# Кодирование категориальных данных
df = pd.get_dummies(df, columns=['ocean_proximity'], drop_first=True)
# Преобразование целевой переменной в категории
df['median_house_value'] = pd.qcut(df['median_house_value'], q=3, labels=[0, 1, 2])

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20433 entries, 0 to 20639
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   longitude                   20433 non-null  float64 
 1   latitude                    20433 non-null  float64 
 2   housing_median_age          20433 non-null  float64 
 3   total_rooms                 20433 non-null  float64 
 4   total_bedrooms              20433 non-null  float64 
 5   population                  20433 non-null  float64 
 6   households                  20433 non-null  float64 
 7   median_income               20433 non-null  float64 
 8   median_house_value          20433 non-null  category
 9   ocean_proximity_INLAND      20433 non-null  bool    
 10  ocean_proximity_ISLAND      20433 non-null  bool    
 11  ocean_proximity_NEAR BAY    20433 non-null  bool    
 12  ocean_proximity_NEAR OCEAN  20433 non-null  bool    
dtypes: bool(4), category(

Сделали один из признаков категориальным, разбили на 3 класса, которые и будем предсказывать

## 2. Провести анализ датасета

### 2.1 Привести описание датасета

Датасет содержит данные о жилых районах в Калифорнии, состоящий из 20640 строк и 10 столбцов. Ниже приведены подробности о каждом столбце:

longitude (float64): Долгота местоположения.

latitude (float64): Широта местоположения.

housing_median_age (float64): Средний возраст домов в районе.

total_rooms (float64): Общее количество комнат в районе.

total_bedrooms (float64): Общее количество спален в районе. (Примечание: содержит 20433 непустых значений из 20640)

population (float64): Численность населения района.

households (float64): Количество домохозяйств в районе.

median_income (float64): Средний доход в районе.

median_house_value (category): Средняя стоимость
домов в районе, представлено 3 классами,
условно: низкая, средня, высокая.

ocean_proximity (object): Близость к океану, категориальный признак.

### 2.2 Масштабирование данных и разбиение выборки

In [54]:
# Масштабирование данных
scaler = MinMaxScaler()
df.loc[:, df.columns != 'median_house_value'] = scaler.fit_transform(df.loc[:, df.columns != 'median_house_value'])

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(df.loc[:, df.columns != 'median_house_value'],
                                                    df['median_house_value'], test_size=0.33, random_state=42)


## 3. Дерево решений

### 3.1 С использованием GridSearchCV осуществить подбор гиперпараметра DecisionTreeClassifier

In [55]:
# Настройка параметров для GridSearchCV
parameters = {
    "criterion": ["gini", "entropy"],
    "splitter": ["best", "random"],
    "max_depth": [None, 10, 20],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ['sqrt', 'log2', None]
}

In [56]:
# Определение оценок для классификации
scores = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score, average='macro'),
    'recall': make_scorer(recall_score, average='macro'),
    'f1': make_scorer(f1_score, average='macro')
}

In [57]:
# Обучение и подбор параметров для DecisionTreeClassifier
clf_tree = GridSearchCV(DecisionTreeClassifier(), parameters, scoring=scores, refit='f1', cv=5, verbose=4)
clf_tree.fit(X_train, y_train)

Fitting 5 folds for each of 324 candidates, totalling 1620 fits
[CV 1/5] END criterion=gini, max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, splitter=best; accuracy: (test=0.717) f1: (test=0.719) precision: (test=0.721) recall: (test=0.717) total time=   0.1s
[CV 2/5] END criterion=gini, max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, splitter=best; accuracy: (test=0.706) f1: (test=0.705) precision: (test=0.704) recall: (test=0.706) total time=   0.1s
[CV 3/5] END criterion=gini, max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, splitter=best; accuracy: (test=0.704) f1: (test=0.704) precision: (test=0.704) recall: (test=0.704) total time=   0.1s
[CV 4/5] END criterion=gini, max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, splitter=best; accuracy: (test=0.698) f1: (test=0.698) precision: (test=0.698) recall: (test=0.698) total time=   0.1s
[CV 5/5] END criterion=gini, max_depth=N

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 10, 20],
                         'max_features': ['sqrt', 'log2', None],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'splitter': ['best', 'random']},
             refit='f1',
             scoring={'accuracy': make_scorer(accuracy_score),
                      'f1': make_scorer(f1_score, average=macro),
                      'precision': make_scorer(precision_score, average=macro),
                      'recall': make_scorer(recall_score, average=macro)},
             verbose=4)

### 3.2 Вывести гиперпараметры лучшей модели и ее метрики

In [58]:
print("Лучшие параметры для DecisionTreeClassifier: ")
print(clf_tree.best_params_)

Лучшие параметры для DecisionTreeClassifier: 
{'criterion': 'gini', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'splitter': 'best'}


In [59]:
results_tree = pd.DataFrame(clf_tree.cv_results_)
best_result_tree = results_tree[results_tree['rank_test_f1'] == 1]
print(f"Accuracy: {best_result_tree['mean_test_accuracy'].values[0]}")
print(f"Precision: {best_result_tree['mean_test_precision'].values[0]}")
print(f"Recall: {best_result_tree['mean_test_recall'].values[0]}")
print(f"F1: {best_result_tree['mean_test_f1'].values[0]}")

Accuracy: 0.7601899196493791
Precision: 0.765313489153228
Recall: 0.760283486275068
F1: 0.7619043418550726


### 3.3 Вывести `feature_importance`

In [60]:
# Важность признаков
feature_importance_tree = {X_train.columns[i]: clf_tree.best_estimator_.feature_importances_[i] for i in range(len(X_train.columns))}
feature_importance_items_tree = sorted(feature_importance_tree.items(), key=lambda x: x[1])
feature_importance_tree = dict(feature_importance_items_tree)
print("Feature importances for DecisionTreeClassifier:")
print(feature_importance_tree)

Feature importances for DecisionTreeClassifier:
{'ocean_proximity_ISLAND': 0.0, 'ocean_proximity_NEAR BAY': 0.0, 'households': 0.01134272389672246, 'ocean_proximity_NEAR OCEAN': 0.013169185774865913, 'total_rooms': 0.015259984922914315, 'total_bedrooms': 0.01695302621555972, 'population': 0.02030797294023238, 'housing_median_age': 0.02747089859008973, 'latitude': 0.13783742994449297, 'longitude': 0.18344119144595508, 'ocean_proximity_INLAND': 0.24477080001012044, 'median_income': 0.32944678625904705}


Как мы видим 2 признака не влияют на среднюю стоимость по району, поэтому дальше мы их убираем, также убираем два наименее значимых признака и берем только 8 лучших признаков

### 3.4. Осуществить фильтрацию признаков

In [61]:
# Выбор наиболее важных признаков
important_features = list(feature_importance_tree.keys())[-8:]  # Топ-8 признаков

In [62]:
# Перетренировка модели с использованием только важных признаков
X_train_f = X_train[important_features]
X_test_f = X_test[important_features]

### 3.5. GridSearch на отфильтрованных данных и вывод лучших гиперпараметров

In [63]:
clf_tree = GridSearchCV(DecisionTreeClassifier(), parameters, scoring=scores, refit='f1', cv=5, verbose=4)
clf_tree.fit(X_train_f, y_train)
print("Лучшие параметры после отбора признаков: ")
print(clf_tree.best_params_)

Fitting 5 folds for each of 324 candidates, totalling 1620 fits
[CV 1/5] END criterion=gini, max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, splitter=best; accuracy: (test=0.711) f1: (test=0.713) precision: (test=0.714) recall: (test=0.711) total time=   0.1s
[CV 2/5] END criterion=gini, max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, splitter=best; accuracy: (test=0.725) f1: (test=0.724) precision: (test=0.724) recall: (test=0.725) total time=   0.1s
[CV 3/5] END criterion=gini, max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, splitter=best; accuracy: (test=0.713) f1: (test=0.712) precision: (test=0.712) recall: (test=0.713) total time=   0.1s
[CV 4/5] END criterion=gini, max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, splitter=best; accuracy: (test=0.693) f1: (test=0.693) precision: (test=0.693) recall: (test=0.693) total time=   0.1s
[CV 5/5] END criterion=gini, max_depth=N

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[CV 1/5] END criterion=gini, max_depth=20, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, splitter=best; accuracy: (test=0.715) f1: (test=0.716) precision: (test=0.717) recall: (test=0.716) total time=   0.1s
[CV 2/5] END criterion=gini, max_depth=20, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, splitter=best; accuracy: (test=0.729) f1: (test=0.729) precision: (test=0.729) recall: (test=0.729) total time=   0.1s
[CV 3/5] END criterion=gini, max_depth=20, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, splitter=best; accuracy: (test=0.691) f1: (test=0.692) precision: (test=0.694) recall: (test=0.691) total time=   0.0s
[CV 4/5] END criterion=gini, max_depth=20, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, splitter=best; accuracy: (test=0.702) f1: (test=0.701) precision: (test=0.701) recall: (test=0.702) total time=   0.1s
[CV 5/5] END criterion=gini, max_depth=20, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, spli

### 3.6. Сравнить метрики

In [64]:
best_result_tree = pd.DataFrame(clf_tree.cv_results_).iloc[clf_tree.best_index_]

In [65]:
print(f"Accuracy: {best_result_tree['mean_test_accuracy']}")
print(f"Precision: {best_result_tree['mean_test_precision']}")
print(f"Recall: {best_result_tree['mean_test_recall']}")
print(f"F1: {best_result_tree['mean_test_f1']}")

Accuracy: 0.7555149744338934
Precision: 0.7605306227377915
Recall: 0.755610068932276
F1: 0.757179413780155


Видно, что метрики оценки качества незначительно уменьшились, зато получилось уменьшить объём данных.

## 4. Случайный лес

### 4.1 С использованием GridSearchCV осуществить подбор гиперпараметра RandomForestClassifier

In [66]:
# Настройка параметров для RandomForestClassifier
parameters_rf = {
    'n_estimators': [50, 100],
    "criterion": ["gini", "entropy"],
    "max_depth": [None, 10],
    "max_features": ['sqrt', 'log2']
}

In [67]:
# Обучение и подбор параметров для RandomForestClassifier
clf_rf = GridSearchCV(RandomForestClassifier(), parameters_rf, scoring=scores, refit='f1', cv=5, verbose=4)
clf_rf.fit(X_train, y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV 1/5] END criterion=gini, max_depth=None, max_features=sqrt, n_estimators=50; accuracy: (test=0.791) f1: (test=0.791) precision: (test=0.792) recall: (test=0.791) total time=   1.3s
[CV 2/5] END criterion=gini, max_depth=None, max_features=sqrt, n_estimators=50; accuracy: (test=0.802) f1: (test=0.802) precision: (test=0.802) recall: (test=0.803) total time=   1.2s
[CV 3/5] END criterion=gini, max_depth=None, max_features=sqrt, n_estimators=50; accuracy: (test=0.789) f1: (test=0.789) precision: (test=0.788) recall: (test=0.789) total time=   1.2s
[CV 4/5] END criterion=gini, max_depth=None, max_features=sqrt, n_estimators=50; accuracy: (test=0.799) f1: (test=0.799) precision: (test=0.799) recall: (test=0.799) total time=   1.2s
[CV 5/5] END criterion=gini, max_depth=None, max_features=sqrt, n_estimators=50; accuracy: (test=0.781) f1: (test=0.781) precision: (test=0.782) recall: (test=0.781) total time=   1.7s
[CV 1/5] END c

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 10],
                         'max_features': ['sqrt', 'log2'],
                         'n_estimators': [50, 100]},
             refit='f1',
             scoring={'accuracy': make_scorer(accuracy_score),
                      'f1': make_scorer(f1_score, average=macro),
                      'precision': make_scorer(precision_score, average=macro),
                      'recall': make_scorer(recall_score, average=macro)},
             verbose=4)

### 4.2 Вывести гиперпараметры лучшей модели и ее метрики

In [68]:
print("Лучшие параметры для RandomForestClassifier: ")
print(clf_rf.best_params_)

Лучшие параметры для RandomForestClassifier: 
{'criterion': 'entropy', 'max_depth': None, 'max_features': 'sqrt', 'n_estimators': 100}


In [69]:
results_rf = pd.DataFrame(clf_rf.cv_results_)
best_result_rf = results_rf[results_rf['rank_test_f1'] == 1]

### 4.3 Осуществить фильтрацию признаков

In [70]:
# Важность признаков
feature_importance_rf = {X_train.columns[i]: clf_rf.best_estimator_.feature_importances_[i] for i in range(len(X_train.columns))}
feature_importance_items_rf = sorted(feature_importance_rf.items(), key=lambda x: x[1])
feature_importance_rf = dict(feature_importance_items_rf)
print("Feature importances for RandomForestClassifier:")
print(feature_importance_rf)

Feature importances for RandomForestClassifier:
{'ocean_proximity_ISLAND': 2.5332867564056996e-05, 'ocean_proximity_NEAR BAY': 0.005064732936989341, 'ocean_proximity_NEAR OCEAN': 0.008984604576501793, 'total_bedrooms': 0.06261681989912016, 'households': 0.06305800536454856, 'housing_median_age': 0.06337007773869226, 'total_rooms': 0.07567618670828948, 'population': 0.07976396644995708, 'ocean_proximity_INLAND': 0.11537719398821107, 'latitude': 0.1354222002115717, 'longitude': 0.14271612515720986, 'median_income': 0.24792475410134485}


In [71]:
# Перетренировка модели с использованием только важных признаков
clf_rf = GridSearchCV(RandomForestClassifier(), parameters_rf, scoring=scores, refit='f1', cv=5, verbose=4)
clf_rf.fit(X_train_f, y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV 1/5] END criterion=gini, max_depth=None, max_features=sqrt, n_estimators=50; accuracy: (test=0.792) f1: (test=0.792) precision: (test=0.793) recall: (test=0.792) total time=   1.1s
[CV 2/5] END criterion=gini, max_depth=None, max_features=sqrt, n_estimators=50; accuracy: (test=0.804) f1: (test=0.803) precision: (test=0.803) recall: (test=0.804) total time=   1.4s
[CV 3/5] END criterion=gini, max_depth=None, max_features=sqrt, n_estimators=50; accuracy: (test=0.789) f1: (test=0.790) precision: (test=0.790) recall: (test=0.789) total time=   1.5s
[CV 4/5] END criterion=gini, max_depth=None, max_features=sqrt, n_estimators=50; accuracy: (test=0.783) f1: (test=0.782) precision: (test=0.782) recall: (test=0.784) total time=   1.2s
[CV 5/5] END criterion=gini, max_depth=None, max_features=sqrt, n_estimators=50; accuracy: (test=0.785) f1: (test=0.786) precision: (test=0.788) recall: (test=0.785) total time=   1.1s
[CV 1/5] END c

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 10],
                         'max_features': ['sqrt', 'log2'],
                         'n_estimators': [50, 100]},
             refit='f1',
             scoring={'accuracy': make_scorer(accuracy_score),
                      'f1': make_scorer(f1_score, average=macro),
                      'precision': make_scorer(precision_score, average=macro),
                      'recall': make_scorer(recall_score, average=macro)},
             verbose=4)

### 4.5 Параметры для лучшей модели

In [72]:
print("Лучшие параметры после отбора признаков: ")
print(clf_rf.best_params_)

Лучшие параметры после отбора признаков: 
{'criterion': 'gini', 'max_depth': None, 'max_features': 'log2', 'n_estimators': 100}


### 4.6 Сравнение метрик до и после фильтрации

In [73]:
print("Метрики до фильтрации: ")
print(f"Accuracy: {best_result_rf['mean_test_accuracy'].values[0]}")
print(f"Precision: {best_result_rf['mean_test_precision'].values[0]}")
print(f"Recall: {best_result_rf['mean_test_recall'].values[0]}")
print(f"F1: {best_result_rf['mean_test_f1'].values[0]}")

Метрики до фильтрации: 
Accuracy: 0.7981008035062089
Precision: 0.7983624052706506
Recall: 0.7982085744741002
F1: 0.7981074175253323


In [74]:
best_result_rf = pd.DataFrame(clf_rf.cv_results_).iloc[clf_rf.best_index_]

print(f"Accuracy: {best_result_rf['mean_test_accuracy']}")
print(f"Precision: {best_result_rf['mean_test_precision']}")
print(f"Recall: {best_result_rf['mean_test_recall']}")
print(f"F1: {best_result_rf['mean_test_f1']}")

Accuracy: 0.8050401753104456
Precision: 0.8059211739355714
Recall: 0.8051359844907953
F1: 0.8053087021913525


Из результатов видно, что значение количественных метрик незначительно, но улучшилось

## 5. Алгоритм K - ближайших соседей

### 5.1 GridSearch для алгоритма

In [76]:
# Настройка параметров для KNeighborsClassifier
parameters_knn = {
    'n_neighbors': [2, 5, 10, 20, 50]
}

In [77]:
# Обучение и подбор параметров для KNeighborsClassifier
clf_knn = GridSearchCV(KNeighborsClassifier(), parameters_knn, scoring=scores, refit='f1', cv=5, verbose=4)
clf_knn.fit(X_train, y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV 1/5] END n_neighbors=2; accuracy: (test=0.693) f1: (test=0.691) precision: (test=0.710) recall: (test=0.693) total time=   0.6s
[CV 2/5] END n_neighbors=2; accuracy: (test=0.696) f1: (test=0.694) precision: (test=0.715) recall: (test=0.697) total time=   0.5s
[CV 3/5] END n_neighbors=2; accuracy: (test=0.693) f1: (test=0.691) precision: (test=0.710) recall: (test=0.693) total time=   0.5s
[CV 4/5] END n_neighbors=2; accuracy: (test=0.687) f1: (test=0.684) precision: (test=0.706) recall: (test=0.687) total time=   0.4s
[CV 5/5] END n_neighbors=2; accuracy: (test=0.686) f1: (test=0.684) precision: (test=0.706) recall: (test=0.687) total time=   0.4s
[CV 1/5] END n_neighbors=5; accuracy: (test=0.715) f1: (test=0.716) precision: (test=0.719) recall: (test=0.716) total time=   0.4s
[CV 2/5] END n_neighbors=5; accuracy: (test=0.730) f1: (test=0.730) precision: (test=0.732) recall: (test=0.730) total time=   0.3s
[CV 3/5] END n_n

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [2, 5, 10, 20, 50]}, refit='f1',
             scoring={'accuracy': make_scorer(accuracy_score),
                      'f1': make_scorer(f1_score, average=macro),
                      'precision': make_scorer(precision_score, average=macro),
                      'recall': make_scorer(recall_score, average=macro)},
             verbose=4)

### 5.2 Лучшие параметры и метрики

In [78]:
print("Лучшие параметры для KNeighborsClassifier: ")
print(clf_knn.best_params_)

Лучшие параметры для KNeighborsClassifier: 
{'n_neighbors': 20}


In [80]:
results_knn = pd.DataFrame(clf_knn.cv_results_)
best_result_knn = results_knn[results_knn['rank_test_f1'] == 1]
print(f"Accuracy: {best_result_knn['mean_test_accuracy'].values[0]}")
print(f"Precision: {best_result_knn['mean_test_precision'].values[0]}")
print(f"Recall: {best_result_knn['mean_test_recall'].values[0]}")
print(f"F1: {best_result_knn['mean_test_f1'].values[0]}")

Accuracy: 0.7287801314828342
Precision: 0.7338756673077464
Recall: 0.7289343191525159
F1: 0.7297914550418051


### 5.3 Обучить на отфильтрованных данных

In [81]:
# Перетренировка модели с использованием только важных признаков
clf_knn.fit(X_train_f, y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV 1/5] END n_neighbors=2; accuracy: (test=0.695) f1: (test=0.693) precision: (test=0.712) recall: (test=0.695) total time=   0.2s
[CV 2/5] END n_neighbors=2; accuracy: (test=0.697) f1: (test=0.694) precision: (test=0.715) recall: (test=0.697) total time=   0.2s
[CV 3/5] END n_neighbors=2; accuracy: (test=0.688) f1: (test=0.685) precision: (test=0.705) recall: (test=0.688) total time=   0.3s
[CV 4/5] END n_neighbors=2; accuracy: (test=0.689) f1: (test=0.686) precision: (test=0.707) recall: (test=0.689) total time=   0.2s
[CV 5/5] END n_neighbors=2; accuracy: (test=0.684) f1: (test=0.681) precision: (test=0.707) recall: (test=0.684) total time=   0.2s
[CV 1/5] END n_neighbors=5; accuracy: (test=0.721) f1: (test=0.721) precision: (test=0.724) recall: (test=0.721) total time=   0.3s
[CV 2/5] END n_neighbors=5; accuracy: (test=0.726) f1: (test=0.726) precision: (test=0.729) recall: (test=0.727) total time=   0.2s
[CV 3/5] END n_n

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [2, 5, 10, 20, 50]}, refit='f1',
             scoring={'accuracy': make_scorer(accuracy_score),
                      'f1': make_scorer(f1_score, average=macro),
                      'precision': make_scorer(precision_score, average=macro),
                      'recall': make_scorer(recall_score, average=macro)},
             verbose=4)

In [82]:
print("Лучшие параметры после отбора признаков: ")
print(clf_knn.best_params_)

Лучшие параметры после отбора признаков: 
{'n_neighbors': 20}


### 5.4 Сравнить метрики для отфильтрованных и неотфильтрованных

In [83]:
print("Метрики до фильтрации: ")
print(f"Accuracy: {best_result_knn['mean_test_accuracy'].values[0]}")
print(f"Precision: {best_result_knn['mean_test_precision'].values[0]}")
print(f"Recall: {best_result_knn['mean_test_recall'].values[0]}")
print(f"F1: {best_result_knn['mean_test_f1'].values[0]}")

Метрики до фильтрации: 
Accuracy: 0.7287801314828342
Precision: 0.7338756673077464
Recall: 0.7289343191525159
F1: 0.7297914550418051


In [84]:
best_result_knn = pd.DataFrame(clf_knn.cv_results_).iloc[clf_knn.best_index_]
print("Метрики после фильтрации: ")
print(f"Accuracy: {best_result_knn['mean_test_accuracy']}")
print(f"Precision: {best_result_knn['mean_test_precision']}")
print(f"Recall: {best_result_knn['mean_test_recall']}")
print(f"F1: {best_result_knn['mean_test_f1']}")

Метрики после фильтрации: 
Accuracy: 0.7285609934258583
Precision: 0.7344933226015379
Recall: 0.7287196678504819
F1: 0.7295929044572114


Из результатов видно, что результаты изменились незнакчительно(менее, чем на 0.1%), однако объём данных уменьшен.

### 6 Оценка качества построенных моделей

In [85]:
# Обучение финальных моделей
knn_model = KNeighborsClassifier(n_neighbors=clf_knn.best_params_['n_neighbors'])
tree_model = DecisionTreeClassifier(**clf_tree.best_params_)
forest_model = RandomForestClassifier(**clf_rf.best_params_)

In [86]:
knn_model.fit(X_train_f, y_train)
tree_model.fit(X_train_f, y_train)
forest_model.fit(X_train_f, y_train)

RandomForestClassifier(max_features='log2')

### 6.1 Визуализировать дерево решений

In [88]:
from sklearn.tree import DecisionTreeRegressor, plot_tree

In [91]:
dot_data = tree.export_graphviz(tree_model, out_file=None,
feature_names=X_train_f.columns,
class_names=['Low', 'Medium', 'High'],
filled=True, rounded=True,
special_characters=True)

graph = graphviz.Source(dot_data)
graph.render("decision_tree") # сохранение в файл
graph.view() # отображение внутри ноутбука

'decision_tree.pdf'

Дерево для удобности просмотра записывается в пдф файл

### 6.2 Сравнить лучшие модели

In [94]:
# Предсказания
y_pred_knn = knn_model.predict(X_test_f)
y_pred_tree = tree_model.predict(X_test_f)
y_pred_forest = forest_model.predict(X_test_f)

print("Результаты для KNeighborsClassifier: ")
print(classification_report(y_test, y_pred_knn))
print("Результаты для DecisionTreeClassifier: ")
print(classification_report(y_test, y_pred_tree))
print("Результаты для RandomForestClassifier: ")
print(classification_report(y_test, y_pred_forest))

Результаты для KNeighborsClassifier: 
              precision    recall  f1-score   support

           0       0.77      0.84      0.80      2263
           1       0.62      0.66      0.64      2241
           2       0.84      0.70      0.76      2239

    accuracy                           0.74      6743
   macro avg       0.74      0.74      0.74      6743
weighted avg       0.74      0.74      0.74      6743

Результаты для DecisionTreeClassifier: 
              precision    recall  f1-score   support

           0       0.82      0.81      0.81      2263
           1       0.64      0.73      0.68      2241
           2       0.86      0.74      0.79      2239

    accuracy                           0.76      6743
   macro avg       0.77      0.76      0.76      6743
weighted avg       0.77      0.76      0.76      6743

Результаты для RandomForestClassifier: 
              precision    recall  f1-score   support

           0       0.84      0.87      0.86      2263
           

Худшая точность у К ближайших соседей, наибольшая точность у случайного леса